- RDDL can be used to model the dynamics of a wildfire and the interventions used to combat it.

- This could include representing the spread of the fire as a stochastic process that evolves over time based on factors such as wind speed and direction, temperature, and the presence of fuel.

- Actions taken by firefighters, such as creating fire breaks, deploying water or chemical retardants, or using bulldozers to remove vegetation, could be represented as interventions that affect the state variables in the model.

- Observations could include information about the location and intensity of the fire, weather conditions, and the effectiveness of interventions.

- By specifying an objective function that takes into account the costs of interventions, the potential damage caused by the fire, and other relevant factors, RDDL could be used to help identify the most effective strategies for fighting the wildfire over time.

- This could involve using techniques such as reinforcement learning to find optimal policies for decision-making in real-time based on the evolving state of the fire and the effectiveness of interventions.

In [ ]:
from pyrddl import Domain, StateVar, Action, Reward

# Define domain
wildfire_domain = Domain(name="wildfire")

# Define state variables
fire_location = StateVar(name="fire_location", values=["north", "south", "east", "west"])
wind_direction = StateVar(name="wind_direction", values=["north", "south", "east", "west"])
temperature = StateVar(name="temperature", values=["low", "medium", "high"])
fuel = StateVar(name="fuel", values=["low", "medium", "high"])
intervention = StateVar(name="intervention", values=["none", "fire_break", "water"])

# Define transition functions
wildfire_domain.add_transition_function(
    name="next_fire_location",
    vars=[fire_location, wind_direction],
    val_fn=lambda fire_location, wind_direction: max(fire_location - 1, "north") if wind_direction == "north" else min(fire_location + 1, "south") if wind_direction == "south" else fire_location
)

wildfire_domain.add_transition_function(
    name="next_temperature",
    vars=[temperature, fire_location, fuel],
    val_fn=lambda temperature, fire_location, fuel: "high" if fire_location == "north" and fuel > "low" else "high" if fire_location == "south" and fuel > "medium" else temperature
)

# Define observation functions
wildfire_domain.add_observation_function(
    name="observe_fire_location",
    vars=[fire_location],
    val_fn=lambda fire_location: fire_location
)

wildfire_domain.add_observation_function(
    name="observe_wind_direction",
    vars=[wind_direction],
    val_fn=lambda wind_direction: wind_direction
)

wildfire_domain.add_observation_function(
    name="observe_temperature",
    vars=[temperature],
    val_fn=lambda temperature: temperature
)

# Define actions
no_intervention = Action(name="no_intervention", param_names=[], precondition=True, effects=[(intervention, "none")])

create_fire_break = Action(name="create_fire_break", param_names=[], precondition=fuel > "medium", effects=[(intervention, "fire_break")])

deploy_water = Action(name="deploy_water", param_names=[], precondition=temperature == "high", effects=[(intervention, "water")])

wildfire_domain.add_actions([no_intervention, create_fire_break, deploy_water])

# Define reward function
wildfire_domain.set_reward(
    Reward(1, [(fire_location, "south"), (intervention, "water")]), # High positive reward for putting out fire
    Reward(-10, [(fire_location, "north"), (intervention, "fire_break")]), # High negative reward for not containing the fire
    default_reward=0 # No reward for other states
)

# Print domain
print(wildfire_domain.to_rddl())


- The first state is "burning" which is represented by the color "flame". This means that the cell is actively on fire and the fire is spreading.

- The second state is "out-of-fuel" which is represented by the color "gray". This means that the cell has already burned and there is no fuel left to sustain the fire.

- The third state is "unburned" which is represented by the color "green". This means that the cell has not yet been affected by the fire and still contains fuel that can ignite and start a fire.

By modeling each cell in this way, the simulation can keep track of which cells are at risk of catching fire, which cells have already burned, and which cells are safe. This information can then be used to make decisions about how to prevent the fire from spreading to important target locations.

#?x

- In RDDL, variables can be either grounded or ungrounded.

- An ungrounded variable is denoted by a question mark (?) followed by a variable name, such as ?x in the code you provided.

- An ungrounded variable is a placeholder for a value that is not known or specified in advance.

- It allows the CPF to be defined in a way that applies to any possible value of the variable.

- When the CPF is used to compute a probability or expected utility, the ungrounded variables will be instantiated with specific values to obtain a concrete value for the probability or expected utility.

In the wildfire RDDL model, the ungrounded variables ?x and ?y represent the spatial coordinates of a cell in the map, and the CPFs use them to calculate the probability of the cell being on fire or out of fuel in the next time step.

For example, if we wanted to calculate the probability that cell (3,4) is on fire in the next time step, we would instantiate ?x and ?y with the values 3 and 4, respectively, and then evaluate the burning' CPF using these values.

The question marks in front of variables such as ?x indicate that they are universally quantified, meaning that the CPF applies to all possible values of those variables.

The first line declares a Conditional Probability Function (CPF) called "burning'" which specifies the probability that a cell is burning in the next time step.

The "if" statement checks whether the cell has been put out or has run out of fuel. If either of these conditions is true, then the cell is no longer burning and its state is set to false.

The "else if" statement checks whether the cell is not out-of-fuel and not already burning. If this condition is true, then it checks if any of the cell's neighbors are burning. If at least one neighbor is burning and the cell is a target, then the cell has a 0% chance of catching fire. Otherwise, the cell has a probability of catching fire based on the function specified inside the "Bernoulli()" function.

The "else" statement simply sets the state of the cell to be the same as its previous state (the state persists).

The second CPF called "out-of-fuel'" specifies the probability that a cell is out of fuel in the next time step.

The "or" operator in the first line checks whether the cell is already out of fuel or is currently burning. If either of these conditions is true, then the cell will remain out of fuel in the next time step.

The "or" operator in the second line checks whether the cell is not a target or has been cut-out. If either of these conditions is true, then the cell will run out of fuel in the next time step.

In summary, these CPFs define the probabilities that a cell will be burning or out-of-fuel in the next time step based on various conditions such as whether the cell is already burning or has been put out, and whether any of its neighbors are burning or out-of-fuel. These probabilities are then used to simulate the spread of a wildfire and the efforts to control it.







he code defines two conditional probability functions (CPFs) for the variables burning and out-of-fuel. These variables represent whether a cell is burning and whether it has run out of fuel, respectively.

The CPF for burning is defined using an if-else statement. If a cell is either put out or has run out of fuel, then it stops burning (i.e., false). Otherwise, if the cell is not out-of-fuel and not already burning, it can potentially ignite into a fire. The probability of this happening depends on the burning status of its neighboring cells, and the function uses a Bernoulli distribution to determine this probability.

The code also includes a check for target cells, which are important locations that need to be protected from the fire. A target cell can only begin to burn if at least one of its neighbors is already burning.

The CPF for out-of-fuel simply sets a cell to be out-of-fuel in the next time step if it is already out-of-fuel, currently burning, or not a target cell and has been cut out. This function does not depend on the state of neighboring cells.

The ' symbol after the variable names indicates that these are the updated variables for the next time step.

Overall, these CPFs are used to model the behavior of a wildfire, taking into account factors such as fuel availability, ignition conditions, and the location of important targets. The goal is to find a sequence of actions that will minimize the expected cost of containing the fire while keeping these target locations safe.




